### Machine Learning

#### Análise do Melhor Modelo


Para identificar o melhor modelo para a previsão de séries temporais, testamos três abordagens distintas:

LSTM (Long Short-Term Memory): Escolhemos o LSTM devido à sua capacidade de capturar dependências temporais de longo prazo e sequenciais, o que é essencial para lidar com dados de séries temporais, onde o padrão ao longo do tempo é crucial para a precisão das previsões.

Gradient Boosting: Este modelo foi testado pela sua robustez e pela habilidade de modelar padrões não lineares e interações complexas em dados tabulares. Sua capacidade de melhorar o desempenho de modelos fracos, combinando várias árvores de decisão, o torna uma excelente opção quando lidamos com múltiplas variáveis e dados heterogêneos.

XGBoost: Optamos também pelo XGBoost, conhecido por sua otimização e alta performance, especialmente quando lidamos com grandes volumes de dados. Além disso, sua capacidade de generalizar bem, mesmo com ajustes finos de parâmetros, é uma característica essencial para garantir precisão e evitar overfitting.

Além disso, implementamos um método para verificar a correlação entre as features. Caso alguma delas apresente uma correlação superior a 90%, removemos essas variáveis para evitar o risco de overfitting, garantindo que o modelo mantenha sua generalização.

Por fim, comparamos os resultados dos três modelos e salvamos aquele que apresentou o melhor desempenho, garantindo assim a escolha da abordagem mais eficaz para nosso problema de previsão.

In [9]:
from datetime import datetime
import os
import pandas as pd
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from sklearn.preprocessing import MinMaxScaler
import joblib
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

def load_and_preprocess_data(url, cutoff_date):
    try:
        tables = pd.read_html(url, decimal=',', thousands='.')
        df = tables[2].drop(index=0).reset_index(drop=True)
        df.columns = ['date', 'price']
        df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
        df.sort_values(by='date', inplace=True)
        df.set_index('date', inplace=True)
        df = df[df.index > cutoff_date].dropna(subset=['price']).reset_index()
        df = create_features(df)
        df = remove_high_correlation(df)
        logging.info(f"Data loaded with {df.shape[0]} records and {df.shape[1]} features.")
        return df
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def create_features(df):
    df['SMA_3'] = df['price'].rolling(window=3).mean()
    df['SMA_6'] = df['price'].rolling(window=6).mean()  
    df['SMA_12'] = df['price'].rolling(window=12).mean() 
    df['pct_change'] = df['price'].pct_change()
    df['lag_1'] = df['price'].shift(1)
    df['lag_3'] = df['price'].shift(3)
    df['lag_6'] = df['price'].shift(6)
    df.dropna(inplace=True)
    return df

def remove_high_correlation(df, threshold=0.9):
    correlation_matrix = df.corr()
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    drop_cols = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
    df = df.drop(columns=drop_cols)
    logging.info(f"Removed {len(drop_cols)} highly correlated features: {drop_cols}")
    return df

def scale_data(df):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df.drop(columns=['date']))
    return scaled_data, scaler

def prepare_ml_data(df, sequence_length):
    scaled_data, scaler = scale_data(df)
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(scaled_data[i + sequence_length][0])
    return np.array(X), np.array(y), scaler

class BaseModel:
    def train(self):
        pass
    
    def predict(self):
        pass
    
    def evaluate(self, y_test, predictions):
        return {
            "r2": r2_score(y_test, predictions),
            "mse": mean_squared_error(y_test, predictions),
            "mae": mean_absolute_error(y_test, predictions),
            "rmse": np.sqrt(mean_squared_error(y_test, predictions))
        }

class LSTMModel(BaseModel):
    def __init__(self, sequence_length, feature_size):
        self.sequence_length = sequence_length
        self.feature_size = feature_size
        self.model = Sequential([ 
            LSTM(50, input_shape=(sequence_length, feature_size)),
            Dense(1)
        ])
        self.model.compile(optimizer='adam', loss='mean_squared_error')
    
    def train(self, X_train, y_train, epochs=100, batch_size=32):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    def predict(self, X_test):
        return self.model.predict(X_test).flatten()

class GradientBoostingModel(BaseModel):
    def __init__(self):
        self.model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05)
    
    def train(self, X_train, y_train):
        self.model.fit(X_train.reshape(X_train.shape[0], -1), y_train)
    
    def predict(self, X_test):
        return self.model.predict(X_test.reshape(X_test.shape[0], -1))

class XGBoostModel(BaseModel):
    def __init__(self):
        self.model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.05)
    
    def train(self, X_train, y_train):
        self.model.fit(X_train.reshape(X_train.shape[0], -1), y_train)
    
    def predict(self, X_test):
        return self.model.predict(X_test.reshape(X_test.shape[0], -1))

def run_pipeline(models, df, sequence_length=10):
    X, y, scaler = prepare_ml_data(df, sequence_length)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    best_model, best_score = None, -np.inf
    for model in models:
        logging.info(f"Training {model.__class__.__name__}...")
        model.train(X_train, y_train)
        predictions = model.predict(X_test)
        metrics = model.evaluate(y_test, predictions)
        logging.info(f"{model.__class__.__name__} - R²: {metrics['r2']:.4f}, RMSE: {metrics['rmse']:.4f}")
        if metrics['r2'] > best_score:
            best_score, best_model = metrics['r2'], model
    return best_model, scaler

def save_model(model, model_name, models_dir):
    if isinstance(model, LSTMModel):
        model.model.save(os.path.join(models_dir, f"{model_name}.keras"))
    else:
        joblib.dump(model.model, os.path.join(models_dir, f"{model_name}.joblib"))

def main():
    url = 'http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view'    
    startDate = f"{datetime.today().year - 10}-{datetime.today().month }-{datetime.today().day}"
    df = load_and_preprocess_data(url, startDate)
    if df is None:
        return
    feature_size = df.shape[1] - 1
    models = [
        LSTMModel(sequence_length=10, feature_size=feature_size),
        GradientBoostingModel(),
        XGBoostModel()
    ]   

    best_model, scaler = run_pipeline(models, df)
    if best_model:
        model_name = best_model.__class__.__name__
        models_dir = os.path.join(os.path.abspath(os.path.pardir), 'models')        
        os.makedirs(models_dir, exist_ok=True)        
        save_model(best_model, model_name, models_dir)        
        joblib.dump(scaler, os.path.join(models_dir, "scaler.joblib"))        
        logging.info(f"Saved best model: {model_name}")

if __name__ == "__main__":
    main()


2025-02-07 12:32:06,121 - INFO - Removed 6 highly correlated features: ['SMA_3', 'SMA_6', 'SMA_12', 'lag_1', 'lag_3', 'lag_6']
2025-02-07 12:32:06,121 - INFO - Data loaded with 2917 records and 3 features.
2025-02-07 12:32:06,163 - INFO - Training LSTMModel...


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


2025-02-07 12:32:41,342 - INFO - LSTMModel - R²: 0.9288, RMSE: 0.0131
2025-02-07 12:32:41,343 - INFO - Training GradientBoostingModel...
2025-02-07 12:32:44,823 - INFO - GradientBoostingModel - R²: 0.9248, RMSE: 0.0134
2025-02-07 12:32:44,824 - INFO - Training XGBoostModel...
2025-02-07 12:32:45,291 - INFO - XGBoostModel - R²: 0.8869, RMSE: 0.0165
2025-02-07 12:32:45,330 - INFO - Saved best model: LSTMModel
